In [72]:
import os
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

from dataset.OnlineDataset import EnergyDataset
import torch

# Gurobi Optimizer

In [9]:
ds = EnergyDataset('../data/1_processed',48,1,'eval')

In [84]:
prosumption = ds[0]['prosumption']
price = ds[0]['price']

In [85]:
for i in range(1,365):
    prosumption= torch.cat([prosumption, ds[i]['prosumption']])
    price = torch.cat([price, ds[i]['price']])

In [86]:
T = len(prosumption)
max_charge = 1.25
max_discharge = -1.25
battery_capacity = 10.0
big_M = 1e4  # A sufficiently large number

# Create model
model = gp.Model("BatteryCharging")
model.Params.OutputFlag = 1

# Variables
action = model.addVars(T, lb=max_discharge, ub=max_charge, name="action")
soe = model.addVars(T + 1, lb=0.0, ub=battery_capacity, name="soe")
z = model.addVars(T, lb=0.0, ub=10, vtype=GRB.CONTINUOUS, name='actual_price')
b = model.addVars(T, vtype=GRB.BINARY, name="b")

# Initial SoE
model.addConstr(soe[0] == 0.0, name="initial_soe")

eps = 0.0001
M = 20 + eps

for t in range(T):
    # Battery dynamics
    model.addConstr(soe[t + 1] == soe[t] + action[t], name=f"soe_update_{t}")
    model.addConstr(action[t]+prosumption[t] >= 0 + eps - M * (1 - b[t]), name="bigM_constr1")
    model.addConstr(action[t]+prosumption[t] <= 0 + M * b[t], name="bigM_constr2")

    # Add indicator constraints
    model.addConstr((b[t] == 1) >> (z[t] == price[t]), name="indicator_constr1")
    model.addConstr((b[t] == 0) >> (z[t] == 0.1), name="indicator_constr2")



# Objective function: minimize cost
obj = gp.quicksum(
    (prosumption[t] + action[t]) * z[t]
    for t in range(T)
)
model.setObjective(obj, GRB.MINIMIZE)

# Optimize
model.optimize()

if model.status == GRB.OPTIMAL:
    final_cost = 0.0
    actions = [action[t].X for t in range(T)]
    soes = [soe[t].X for t in range(T+1)]
    used_price = [z[t].X for t in range(T)]
    for t in range(T):
        curr_price = price[t] if (prosumption[t]+actions[t]) >= 0 else 0.1
        final_cost += (prosumption[t]+actions[t])*curr_price

    print(f'final cost: {final_cost}')
    print(f'net: {[float(prosumption[t])+actions[t] for t in range(T)]}')
    print(used_price)
    print(soes)
    print(actions)
else:
    print("Optimization failed.")
    # return None, None


model.IsMIP

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 3700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 52561 rows, 70081 columns and 122641 nonzeros
Model fingerprint: 0xf8ef897d
Model has 17520 quadratic objective terms
Model has 35040 simple general constraints
  35040 INDICATOR
Variable types: 52561 continuous, 17520 integer (17520 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-03, 3e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [4e-03, 2e+01]
  GenCon rhs range [3e-02, 4e-01]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 4305 rows and 19675 columns
Presolve time: 0.71s
Presolved: 63625 rows, 65775 columns, 160137 nonzeros
Variable types: 50406 continuous, 15369 integer (15369 binary)
Performing ano

KeyboardInterrupt: 

Exception ignored in: 'gurobipy._core.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\pasca\miniconda3\envs\dtkd\Lib\site-packages\IPython\core\interactiveshell.py", line 3043, in write
    def write(data, *args, **kwargs):
KeyboardInterrupt: 


  1751  1269  748.06412   39 1533  748.09991  748.02141  0.01%   1.9   66s
  1756  1273  748.08981   58   36  748.09991  748.02141  0.01%   1.9   70s

Cutting planes:
  Learned: 10410
  Gomory: 55
  Implied bound: 191
  Projected implied bound: 7
  MIR: 11057
  Mixing: 4
  Flow cover: 214
  RLT: 3

Explored 1756 nodes (175206 simplex iterations) in 70.25 seconds (39.61 work units)
Thread count was 16 (of 16 available processors)

Solution count 10: 748.1 748.108 748.129 ... 748.138

Optimal solution found (tolerance 1.00e-04)
Best objective 7.480999092994e+02, best bound 7.480441904441e+02, gap 0.0074%
final cost: 748.0979614257812
net: [2.9639999866485596, 0.1239999532699585, 1.2380000352859497, 0.09399999678134918, 1.3880000114440918, 0.0, 1.7369999885559082, 1.3379999995231628, 2.0830000042915344, 1.8029999732971191, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.999999999976694e-05, 9.999999999976694e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

1